## RF. Performance assessment.


## Import Libraries:

In [93]:
import numpy as np
import pandas as pd
import datetime
from pathlib import Path

import pickle

import sys
sys.path.append( '..')


In [94]:
layer = 'RNFL' # 'RNFL' # 'GCL'
eye = 'Random' # 'L', 'R', 'Both', 'Random'

In [95]:
df = pd.read_csv( '../../Bases_Datos_Tabulares_Elena_Nov2022/BaseDatos_PPole_EM_2020.csv' )    

In [96]:
# Count

def count( df, g ):
    
    dfb = df.copy()
    
    df = df[df['GRUPO (0=Sano, 1=EM)'] == g]
    df = df[df['OJO (D=1, I=2)']== 1]

    print( 'R')

    print(df.shape)
    print(df.groupby('APELLIDOS').head(1).shape)

    df = dfb
    
    df = df[df['GRUPO (0=Sano, 1=EM)'] == g]
    df = df[df['OJO (D=1, I=2)']== 2]


    print( 'L')

    print(df.shape)
    print(df.groupby('APELLIDOS').head(1).shape)

In [97]:
count(df, 0)
count(df, 1)

R
(34, 159)
(32, 159)
L
(32, 159)
(31, 159)
R
(50, 159)
(46, 159)
L
(50, 159)
(47, 159)


In [98]:
import re

def RNFL_features(df, features):
    
    regex = re.compile( 'Mean [0-9].[0-9] RNFL' )
    filtered = [f for f in features if regex.match(f)]

    for f in filtered:
        df[f] = pd.to_numeric(df[f], errors='coerce')
        
    print(df.shape)

    df = df.dropna(subset = filtered, how = 'all') 

    df = df.fillna(0)

    print(df.shape)

    return df, filtered


def GCL_features(df, features):
    
    regex = re.compile( 'Mean [0-9].[0-9] GCL' )
    filtered = [f for f in features if regex.match(f)]

    for f in filtered:
        df[f] = pd.to_numeric(df[f], errors='coerce')
        
    print(df.shape)

    df = df.dropna(subset = filtered, how = 'all') 

    df = df.fillna(0)

    print(df.shape)

    return df, filtered

In [99]:
features = df.keys()

if layer == 'RNFL':    
    df, filtered = RNFL_features(df, features)
elif layer == 'GCL':
    df, filtered = GCL_features(df, features)


(166, 159)
(166, 159)


In [100]:
count(df, 0)
count(df, 1)

R
(34, 159)
(32, 159)
L
(32, 159)
(31, 159)
R
(50, 159)
(46, 159)
L
(50, 159)
(47, 159)


In [101]:
if eye == 'R':
    df = df[df['OJO (D=1, I=2)'] == 1]
elif eye == 'L':
    df = df[df['OJO (D=1, I=2)'] == 2]
elif eye == 'Both':
    print( 'Nothing' )
elif eye == 'Random':

    patients = df['APELLIDOS'].unique()
    
    indexes = []
    
    for p in patients:
        
        r = randint(1, 2)
            
        record = df[df['APELLIDOS'] == p]
        
        if record.shape[0] > 1:
                    
            record = record[record['OJO (D=1, I=2)'] == r]
            
            if record.shape[0] > 2:
                print(record)
            
        indexes.append(record.index.values)
            
    df = df.loc[list(itertools.chain(*indexes))]

In [102]:
sanos = df 
sanos = sanos[sanos['GRUPO (0=Sano, 1=EM)'] == 0]

em = df 
em = em[em['GRUPO (0=Sano, 1=EM)'] == 1]

In [103]:
from random import randint

import itertools

def select_train_test( df, test_subjects ):

    patients = df['APELLIDOS'].unique()

    indexes = []
    
    for p in patients:
        
        if p not in test_subjects:
            
            record = df[df['APELLIDOS'] == p]
        
            indexes.append(record.index.values)
            
    train = df.loc[list(itertools.chain(*indexes))]
    
    indexes = []

    for p in patients:
        
        if p in test_subjects:
        
            record = df[df['APELLIDOS'] == p]
        
            indexes.append(record.index.values)
    
    test = df.loc[list(itertools.chain(*indexes))]
    
    return train, test


In [104]:
import imblearn
print(imblearn.__version__)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

patients = df['APELLIDOS'].unique()

prediction = []

for p in patients:
    
    print( "Processing patient ", p )
    
    test_subjects = p
    
    em_train, em_test = select_train_test(em, test_subjects)
    sanos_train, sanos_test = select_train_test(sanos, test_subjects)
    
    print(em_train.shape)
    print(sanos_train.shape)

    print(em_test.shape)
    print(sanos_test.shape)
    
    filtered = list(filtered) # + ['Lastname']
    
    em_train = em_train[filtered]
    sanos_train = sanos_train[filtered]

    em_test = em_test[filtered]
    sanos_test = sanos_test[filtered]

    em_train['Diagnosis'] = 1
    sanos_train['Diagnosis'] = 0

    em_test['Diagnosis'] = 1
    sanos_test['Diagnosis'] = 0
    
    ddf = em_train.append(sanos_train)

    y_train = ddf['Diagnosis']
    train = ddf.drop('Diagnosis', axis=1)

    ddf = em_test.append(sanos_test)

    y_test = ddf['Diagnosis']
    test = ddf.drop('Diagnosis', axis=1)
    

    # transform the dataset
    counter = Counter(y_train)
    print(counter)

    oversample = SMOTE()
    train, y_train = oversample.fit_resample(train, y_train)

    counter = Counter(y_train)
    print(counter)

    # --

    # Default parameters

    model = RandomForestClassifier()

    model.fit( train, y_train )
    
    classification = model.predict_proba(test)
    
    ntest = test.shape[0]
    hardEstimClass = -1 * np.ones(ntest, int)

    for s in range(ntest):
        hardEstimClass[s] = np.argmax([classification[s][0], classification[s][1]])
        
    prediction.append( [p, classification, hardEstimClass[0], df[df['APELLIDOS']== p]['GRUPO (0=Sano, 1=EM)'].values[0]] )
    

0.8.1
Processing patient  CORDON
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  BLASCO
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  GIL GOMEZ
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  ABADIA
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  HERNANDEZ VIAN
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  FUERTES LAZARO
(63, 159)
(33, 159)
(0, 159)
(2, 159)
Counter({1: 63, 0: 33})
Counter({1: 63, 0: 63})
Processing patient  DIAZ PEIRO
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  CALVO ROYO
(63, 159)
(34, 159)
(0, 159)
(1, 159)
Counter({1: 63, 0: 34})
Counter({1: 63, 0: 63})
Processing patient  RUIZ ORTIZ
(63, 159)
(34, 159)
(0, 159)
(1,

(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  SANZ PARDILLOS
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  PORTAL HERNANDEZ
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  TEJEDOR MARIN
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  BENEDI ESPAÑA
(61, 159)
(35, 159)
(2, 159)
(0, 159)
Counter({1: 61, 0: 35})
Counter({1: 61, 0: 61})
Processing patient  PINA VIDAL
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  CATALAN
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  CALLE GIMENO
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Counter({1: 62, 0: 35})
Counter({1: 62, 0: 62})
Processing patient  LAZARO HERNANDEZ
(62, 159)
(35, 159)
(1, 159)
(0, 159)
Count

In [105]:
from BCA import calcBCA
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report

estimLabels = []
trueLabels = []

# https://uberpython.wordpress.com/2012/01/01/precision-recall-sensitivity-and-specificity/
for i in range(len(prediction)):
    estimLabels += [prediction[i][2]]
    trueLabels += [prediction[i][3]]

estimLabels = np.array(estimLabels)    
trueLabels = np.array(trueLabels)    

calcBCA(estimLabels, trueLabels, 2)

# https://stackoverflow.com/questions/58373488/why-does-my-sklearn-metrics-confusion-matrix-output-look-transposed
conf = confusion_matrix(trueLabels, estimLabels, [0,1] )

# Estoy con la clase 1
tp = conf[1][1]
tn = conf[0][0]
fp = conf[0][1]
fn = conf[1][0]

prec = precision_score(trueLabels, estimLabels, [1])

recall = recall_score(trueLabels, estimLabels, [1])

acc = accuracy_score(trueLabels, estimLabels, [1])

bca = balanced_accuracy_score(trueLabels, estimLabels)

f1 = f1_score(trueLabels, estimLabels, [1])

auc = roc_auc_score(trueLabels, estimLabels )

sensitivity = (1. * tp)/(tp+fn)
specificity = (1. * tn)/(tn+fp)


print( "Confusion" )
print(conf)

print( "prec, recall, acc, bca")
print(prec)
print(recall)
print(acc)
print(bca)

print( "f1, auc")
print(f1)
print(auc)
 
print(sensitivity, specificity)

print(classification_report(estimLabels, trueLabels, [0,1]))


"""
sensitivity = recall = tp / t = tp / (tp + fn)
specificity = tn / n = tn / (tn + fp)
precision = tp / p = tp / (tp + fp)
"""

"""
Formato para latex:  Accuracy (\%) & Sensitivity (\%) & Specificity (\%) & F1-score (\%) & AUC 
"""

print( layer, " & ", "{:.2f}".format(acc*100), " & ", "{:.2f}".format(sensitivity*100), " & ", "{:.2f}".format(specificity*100), " & ", "{:.2f}".format(f1*100), " & ", "{:.2f}".format(auc*100) )

Class 0 bcaCurr 0.673720 TP 20.000000 TN 43.000000 FP 15.000000 FN 13.000000 acc 0.692308 sens 0.606061 spec 0.741379
Class 1 bcaCurr 0.673720 TP 43.000000 TN 20.000000 FP 13.000000 FN 15.000000 acc 0.692308 sens 0.741379 spec 0.606061
Confusion
[[20 13]
 [15 43]]
prec, recall, acc, bca
0.7678571428571429
0.7413793103448276
0.6923076923076923
0.6737199582027169
f1, auc
0.7543859649122806
0.6737199582027169
0.7413793103448276 0.6060606060606061
              precision    recall  f1-score   support

           0       0.61      0.57      0.59        35
           1       0.74      0.77      0.75        56

    accuracy                           0.69        91
   macro avg       0.67      0.67      0.67        91
weighted avg       0.69      0.69      0.69        91

RNFL  &  69.23  &  74.14  &  60.61  &  75.44  &  67.37


/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[0, 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass normalize=[1] as keywor